# Prepration
Loading env variables and vectorDB

In [1]:
from dotenv import load_dotenv
import os
import pickle

load_dotenv()

True

In [2]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

/Users/atorres/.local/share/virtualenvs/langchain-kv9OLnAB-python/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Agents

Agents use an LLM to determine which actions to perform and in specific order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

## Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents.

In [10]:
from langchain.agents import load_tools
from langchain.llms import Replicate

llm = Replicate(
    model="replicate/llama-2-70b-chat:2c1608e18606fad2812020dc541930f2d0495ce32eee50074220b87300bc16e1",
    input={"temperature": 0.9, "max_length":100},
)


tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion:

In [11]:
from langchain.agents import Tool
tool_list = [
    Tool(
        name = "Math Tool",
        func=tools[0].run,
        description="Tool to calculate mathematical operations, nothing else"
    )
]

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(tool_list, 
                         llm, 
                         agent="zero-shot-react-description",
                         verbose=True)
agent.run("How are you?")

In [ ]:
agent.run("What is 100 divided by 25? just say the number")

# Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool.

In [19]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "Restaurant search"
    description = "useful for when you need to answer questions about the restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [20]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("When does the restaurant open? just return the time.") 